In [1]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from tailnflows.utils import get_project_root, load_raw_data

In [3]:
losses = load_raw_data('2024-05-synth-de-dry_losses')

{label: [
    train_data['tst_ix'] for train_data in data
] for label, data in losses.items()}

{'normal': [-1, 9, 8],
 'ttf_m': [-1, 9, 9],
 'ttf_m_fix': [-1, 8, 9],
 'mtaf': [-1, 5, 8],
 'gtaf': [-1, 9, 9],
 'm_normal': [-1, 9],
 'g_normal': [-1, 9],
 'comet': [9]}

In [46]:
# raw_data = load_raw_data('alt_shift_de_2024-02-07')
raw_data = load_raw_data('alt_shift_de_2023_12_01')
raw_data.keys()

dict_keys(['g_normal', 'm_normal', 'gTAF', 'mTAF', 'TTF_m', 'TTF_m_fixed', 'comet', 'normal', 'TTF_m_random_init'])

In [40]:
raw_data

{}

In [49]:
raw_data = load_raw_data('alt_shift_de_2024-02-08')
rows = []
for label, _data in raw_data.items():
    # if label != 'comet':
    for test_ll, dim, df in _data:
        rows.append({
            'model': label,
            'tst_negll_per_dim': test_ll / dim,
            'df': df,
            'dim': dim
        })


raw_data = load_raw_data('alt_shift_de_2023_12_01')
for test_ll, dim, df in raw_data['comet']:
    rows.append({
        'model': 'comet',
        'tst_negll_per_dim': test_ll / dim,
        'df': df,
        'dim': dim
    })

In [52]:
results = pd.DataFrame(rows)
results = results[~results.isin([np.nan, np.inf, -np.inf]).any(1)]


/tmp/ipykernel_8540/989590601.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  results = results[~results.isin([np.nan, np.inf, -np.inf]).any(1)]


In [53]:
results.groupby(['model', 'df', 'dim']).count()

tst_negll_per_dim
model        df   dim                   
TTF_m_fixed  0.5  5                    5
                  10                   5
                  50                   5
             1.0  5                    5
                  10                   5
                  50                   5
             2.0  5                    5
                  10                   5
                  50                   5
             30.0 5                    5
                  10                   5
                  50                   5
TTF_m_stable 0.5  5                    5
                  10                   5
                  50                   5
             1.0  5                    5
                  10                   5
                  50                   5
             2.0  5                    5
                  10                   5
                  50                   5
             30.0 5                    5
                  10                   5
                  50                   5
comet        0.5  5                    7
             1.0  5                    7
             1.5  5                    7
             2.0  5                    7
             10.0 5                    7
             30.0 5                    7
gTAF         0.5  5                    5
                  10                   5
                  50                   5
             1.0  5                    5
                  10                   5
                  50                   5
             2.0  5                    5
                  10                   5
                  50                   5
             30.0 5                    5
                  10                   5
                  50                   3
g_normal     0.5  5                    1
mTAF         0.5  5                    5
                  10                   5
                  50                   5
             1.0  5                    5
                  10                   5
                  50                   5
             2.0  5                    5
                  10                   5
                  50                   5
             30.0 5                    5
                  10                   5
                  50                   5

In [73]:
np.logical_and(
    results['model'] == 'TTF_m_fixed',
    results['dim'] == 5,
    results['df'] == 0.5,
).sum()

20

In [68]:
results[np.logical_and(
    results['model'] == 'TTF_m_fixed',
    results['dim'] == 5,
    results['df'] == 0.5,
)]

,model,tst_negll_per_dim,df,dim
60,TTF_m_fixed,3.504910,0.5,5
61,TTF_m_fixed,2.429257,1.0,5
62,TTF_m_fixed,2.020833,2.0,5
63,TTF_m_fixed,1.567395,30.0,5
72,TTF_m_fixed,3.506306,0.5,5
73,TTF_m_fixed,2.480242,1.0,5
74,TTF_m_fixed,2.002068,2.0,5
75,TTF_m_fixed,1.552183,30.0,5
84,TTF_m_fixed,3.461781,0.5,5
85,TTF_m_fixed,2.473493,1.0,5


### Box Plots

In [54]:
import seaborn as sns

wanted_dims = sorted(results['dim'].unique())
wanted_dfs = sorted(results['df'].unique())
wanted_models = results.model.unique()

fig, axarr = plt.subplots(len(wanted_dims), len(wanted_dfs), figsize=(16, 12), tight_layout=True)

if len(wanted_dims) == 1 and len(wanted_dfs) == 1:
  axarr = [[axarr]]

sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.)

for i, dim in enumerate(wanted_dims):
  for j, nuisance_df in enumerate(wanted_dfs):
    wanted_data = results[np.logical_and(
        results['dim'] == dim,
        results['df'] == nuisance_df,
    )]

    sns.boxplot(
        data=wanted_data,
        x='tst_negll_per_dim',
        y='model',
        order=wanted_models,
        orient='h',
        ax=axarr[i][j],
    )

    axarr[i][j].set_title(f'dim={dim} df={nuisance_df:.1f}')

    # if j == 0:
      # axarr[i][j].set_xlim([0., 2.3])

Error in callback <function flush_figures at 0x7f2a9254d310> (for post_execute):


KeyboardInterrupt: 

In [56]:
agg = results.groupby(['dim', 'df', 'model'], group_keys=False).agg(['mean', 'sem'])

unpacked_rows = []
for (dim, df, model), test_neg_ll_agg in agg.iterrows():
    unpacked_rows.append(
        {
            'model': model,
            'df': df,
            'dim': dim,
            'tst_negll': test_neg_ll_agg.values[0],
            'sem': test_neg_ll_agg.values[1],
        }
    )

rezzas = pd.DataFrame(unpacked_rows)
rezzas

,model,df,dim,tst_negll,sem
0,TTF_m_fixed,0.5,5,3.487920e+00,0.009735
1,TTF_m_stable,0.5,5,3.514610e+00,0.013798
2,comet,0.5,5,3.299556e+00,0.015982
3,gTAF,0.5,5,5.322728e+00,0.127759
4,g_normal,0.5,5,5.108890e+10,NaN
5,mTAF,0.5,5,4.007118e+00,0.029783
6,TTF_m_fixed,1.0,5,2.462721e+00,0.009938
7,TTF_m_stable,1.0,5,2.474672e+00,0.005437
8,comet,1.0,5,2.332480e+00,0.006649
9,gTAF,1.0,5,2.642197e+00,0.022110


In [57]:
def readable(row):
    if row["tst_negll"] > 10000 or np.isnan(row["tst_negll"]):
        return 'null'
    else:
        return  f'{row["tst_negll"]:.2f} ({row["sem"]:.2f})'
    
def interval(row):
    if row["tst_negll"] > 10000 or np.isnan(row["tst_negll"]):
        return 'null'
    else:
        return  f'[{row["tst_negll"] - row["sem"]:.4f}, {row["tst_negll"] + row["sem"]:.4f}]'
    
rezzas['readable'] = rezzas.apply(readable, axis=1)
rezzas['interval'] = rezzas.apply(interval, axis=1)

selector = np.logical_and(
    rezzas['dim'] == 5,
    np.isin(
        rezzas['df'], [0.5, 1., 2., 30.]
    ),
    # np.isin(
    #     rezzas['model'],
    #     ['TTF_m_affine_stable', 'TTF_m_stable']
    # )
)

res_tab = rezzas[selector].pivot(index='model', columns='df')['readable']
res_tab

df,0.5,1.0,2.0,30.0
model,,,,
TTF_m_fixed,3.49 (0.01),2.46 (0.01),2.00 (0.01),1.56 (0.00)
TTF_m_stable,3.51 (0.01),2.47 (0.01),2.00 (0.01),1.58 (0.01)
comet,3.30 (0.02),2.33 (0.01),1.87 (0.00),1.46 (0.00)
gTAF,5.32 (0.13),2.64 (0.02),2.07 (0.01),1.57 (0.00)
g_normal,null,NaN,NaN,NaN
mTAF,4.01 (0.03),2.60 (0.01),2.05 (0.01),1.55 (0.01)


In [58]:
order = {
    # 'normal': 'normal',
    # 'm_normal': 'm\_normal',
    # 'g_normal': 'g\_normal',
    # 'gTAF': 'gTAF',
    # 'TTF_m_affine_stable': 'TTF',
    # 'mTAF': 'mTAF',
    # 'TTF_m_affine_fixed': 'TTF (fix)',
    'comet': 'COMET'
}

for m_name, l_name in order.items():
    parsed_vals= [
        val if isinstance(val, str) else 'null'
        for val in res_tab.loc[m_name]
    ]
    print(f'& {l_name} &', ' & '.join(parsed_vals), ' \\\\')

& COMET & 3.30 (0.02) & 2.33 (0.01) & 1.87 (0.00) & 1.46 (0.00)  \\


In [16]:

for m_name, l_name in order.items():
    parsed_vals= [
        val if isinstance(val, str) else 'null'
        for val in res_tab.loc[m_name]
    ]
    print(f'& {l_name} &', ' & '.join(parsed_vals), ' \\\\')

KeyError: 'normal'